<a href="https://colab.research.google.com/github/Ign0r3dH4x0r/Torrent2drive/blob/master/torrent-to-gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IGN0R3DH4X0R TORRENT2DRIVE

**TIPS!** GET MORE DISK SPACE:
> RUNTIME => CHANGE RUNTIME => SELECT GPU => GET ~ 350 GB

### INSTALL REQUIREMENTS

In [4]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

     |████████████████████████████████| 1.5MB 8.3MB/s 
     |████████████████████████████████| 788kB 39.3MB/s 
Requirement already up-to-date: wheel in /usr/local/lib/python3.7/dist-packages (0.36.2)
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Found existing installation: setuptools 54.0.0
    Uninstalling setuptools-54.0.0:
      Successfully uninstalled setuptools-54.0.0
     |████████████████████████████████| 2.4 MB 7.5 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-libtorrent is already the newest version (1.1.5-1build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


### MOUNT DRIVE
TO DOWNLOAD TORRENT, GIVE ACCESS.

In [2]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


### ADD FROM FILE
You can run this cell to add more files as many times as you want

In [3]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/IGN0R3DH4X0R_Torrents",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

Saving Udemy - Website Hacking, Penetration Testing & Bug Bounty Hunting.torrent to Udemy - Website Hacking, Penetration Testing & Bug Bounty Hunting.torrent


NameError: ignored

### ADD FROM MAGNET LINK
You can run this cell to add more files as many times as you want

In [ ]:
params = {"save_path": "/content/drive/My Drive/IGN0R3DH4X0R_Torrents"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


### START DOWNLOAD
DOWNLOAD TORRENTS.

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)
